In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from functools import reduce
from sklearn.model_selection import KFold
from sklearn.svm import SVC 
from sklearn.metrics import roc_auc_score 

#### data split

In [8]:
def data_reorder(data):
    cols = data.index.tolist() 
    sorted_cols = [] 
    for col in cols:
        if 'T2D' in col:
            sorted_cols.insert(0, col) 
        else:
            sorted_cols.append(col) 
    new_data = data.reindex(index=sorted_cols)
    new_data = new_data.T
    return new_data

In [9]:
GSE76894_exp = pd.read_csv("/home/xqxie/Task/02_stable_reverse_gene_pair/6_ESWA_review/GSE76894_exp.csv",index_col=0)
GSE164416_exp = pd.read_csv("/home/xqxie/Task/02_stable_reverse_gene_pair/6_ESWA_review/GSE164416_exp.csv",index_col=0)
GSE54279_exp = pd.read_csv("/home/xqxie/Task/02_stable_reverse_gene_pair/6_ESWA_review/GSE54279_exp.csv",index_col=0)

In [10]:
GSE76894_exp

,T2D_1,T2D_2,T2D_3,T2D_4,T2D_5,T2D_6,T2D_7,T2D_8,T2D_9,T2D_10,...,ND_75,ND_76,ND_77,ND_78,ND_79,ND_80,ND_81,ND_82,ND_83,ND_84
RPL24,14.490839,14.468191,14.460272,14.326163,14.516077,14.478849,14.488322,14.443283,14.444448,14.481314,...,14.404313,14.494281,14.479684,14.478916,14.359072,14.487879,14.487879,14.369230,14.480880,14.501024
PRSS2,14.468511,14.416742,14.004867,14.385120,14.416041,14.433298,14.248996,14.434843,14.408695,14.392272,...,14.414484,14.443124,14.431339,14.472357,14.406583,14.253243,14.345155,14.454317,14.428639,14.435552
RPL41,14.339868,14.389378,14.355241,14.285235,14.390659,14.400078,14.367675,14.353179,14.353914,14.358337,...,14.324145,14.369616,14.363254,14.380254,14.338537,14.391413,14.428541,14.314028,14.376708,14.382131
RPL37A,14.351845,14.314872,14.250718,14.356604,14.367312,14.354591,14.381406,14.368568,14.366039,14.389377,...,14.311746,14.373850,14.401507,14.346892,14.404702,14.305756,14.327023,14.324626,14.373151,14.341500
FTH1,14.352249,14.359718,14.382342,14.369362,14.172053,14.273790,14.410985,14.342881,14.249919,14.272677,...,14.305274,14.296401,14.290064,14.349175,14.350673,14.188318,14.211825,14.358854,14.315529,14.297553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOP2A,4.617550,5.498190,7.393819,4.382055,4.026597,4.541374,7.550514,5.023112,4.116082,3.865579,...,5.429609,5.514477,5.006005,4.800530,5.776840,6.906264,4.862126,3.572200,6.740960,4.558207
LRRC19,3.742579,4.928037,7.292803,5.461342,5.096469,3.473372,3.527970,6.048153,6.604290,4.336879,...,6.751697,4.860942,3.085955,6.056345,4.060692,5.009601,3.608305,4.541697,6.191811,4.251162
UBE2U,7.820590,8.086299,7.366528,4.128568,4.207169,6.021596,5.228401,5.130074,4.400424,6.245050,...,3.685759,4.539017,7.031129,7.268719,3.294204,6.179686,6.300010,3.050115,6.770793,4.173458
GUCA1C,5.483316,5.496380,3.576556,5.442962,6.154804,5.232584,4.678285,4.947814,4.946902,3.086746,...,5.277346,5.884604,6.918855,6.392963,3.422660,3.689710,3.971264,5.744106,6.118661,6.813979


In [11]:
GSE76894_label = [1 for index in range(19)] + [0 for index in range(84)]
GSE76894_train, GSE76894_test = train_test_split(GSE76894_exp.T, test_size=0.3, random_state=42, stratify=GSE76894_label)
GSE76894_train = data_reorder(GSE76894_train)
GSE76894_test = data_reorder(GSE76894_test)

GSE164416_label = [1 for index in range(39)] + [0 for index in range(18)]
GSE164416_train, GSE164416_test = train_test_split(GSE164416_exp.T, test_size=0.3, random_state=42, stratify=GSE164416_label)
GSE164416_train = data_reorder(GSE164416_train)
GSE164416_test =  data_reorder(GSE164416_test)

GSE54279_label = [1 for index in range(16)] + [0 for index in range(44)]
GSE54279_train, GSE54279_test = train_test_split(GSE54279_exp.T, test_size=0.3, random_state=42, stratify=GSE54279_label)
GSE54279_train = data_reorder(GSE54279_train)
GSE54279_test =  data_reorder(GSE54279_test)

GSE76894_train

,T2D_7,T2D_2,T2D_14,T2D_10,T2D_5,T2D_11,T2D_16,T2D_3,T2D_12,T2D_4,...,ND_40,ND_83,ND_67,ND_65,ND_48,ND_11,ND_46,ND_54,ND_13,ND_17
RPL24,14.488322,14.468191,14.494157,14.481314,14.516077,14.504426,14.418090,14.460272,14.391797,14.326163,...,14.454460,14.480880,14.431810,14.404690,14.392974,14.336503,14.469028,14.450462,14.426674,14.491541
PRSS2,14.248996,14.416742,14.569248,14.392272,14.416041,14.416262,14.392535,14.004867,14.336725,14.385120,...,14.462269,14.428639,14.310074,14.353287,14.450116,14.368102,14.388319,14.327300,14.481665,14.321064
RPL41,14.367675,14.389378,14.321896,14.358337,14.390659,14.350549,14.355957,14.355241,14.305650,14.285235,...,14.381443,14.376708,14.401245,14.391086,14.313863,14.292106,14.362015,14.343400,14.375433,14.328203
RPL37A,14.381406,14.314872,14.353214,14.389377,14.367312,14.332060,14.316136,14.250718,14.345119,14.356604,...,14.361536,14.373151,14.270924,14.375428,14.271798,14.364153,14.322187,14.327283,14.332962,14.361236
FTH1,14.410985,14.359718,14.337550,14.272677,14.172053,14.360151,14.341013,14.382342,14.321036,14.369362,...,14.368653,14.315529,14.178374,14.228469,14.305223,14.388207,14.273223,14.378753,14.263925,14.328506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOP2A,7.550514,5.498190,4.553856,3.865579,4.026597,5.496223,6.541450,7.393819,7.385488,4.382055,...,3.590700,6.740960,4.030822,4.596963,5.657742,4.585799,3.833329,5.444821,4.011721,6.685705
LRRC19,3.527970,4.928037,4.628409,4.336879,5.096469,5.957162,5.577122,7.292803,4.369765,5.461342,...,3.990002,6.191811,4.910182,6.675582,5.895703,6.006509,7.189799,7.725888,5.154836,7.255313
UBE2U,5.228401,8.086299,4.185913,6.245050,4.207169,4.996908,6.003644,7.366528,4.996908,4.128568,...,3.516434,6.770793,5.806270,4.969562,7.869864,4.905539,6.350919,4.687487,5.059856,6.650969
GUCA1C,4.678285,5.496380,5.002247,3.086746,6.154804,4.598857,4.705845,3.576556,4.978671,5.442962,...,3.769780,6.118661,4.513871,6.075469,4.485564,6.534073,6.208248,4.381535,4.342595,4.645615


#### reverse REOs extract -- 65%

In [42]:
def t2d_matrix(x,t2d_exp,threshold):
    sample_num = t2d_exp.shape[1]
    diff = x - t2d_exp
    diff_po = diff > 0
    diff_ne = diff < 0
    diff_po_num = diff_po.sum(axis=1)  
    diff_ne_num = diff_ne.sum(axis=1)
    # set the threshold for obtaining REOs 
    diff_po_re = (diff_po_num > (threshold * sample_num)) * 1
    diff_ne_re = (diff_ne_num > (threshold * sample_num)) * -1
    return (diff_po_re + diff_ne_re)

def nd_matrix(x,nd_exp,threshold):
    sample_num = nd_exp.shape[1]
    diff = x - nd_exp
    diff_po = diff > 0
    diff_ne = diff < 0
    diff_po_num = diff_po.sum(axis=1)  
    diff_ne_num = diff_ne.sum(axis=1)
    # set the threshold for obtaining REOs 
    diff_po_re = (diff_po_num > (threshold * sample_num)) * 1
    diff_ne_re = (diff_ne_num > (threshold * sample_num)) * -1
    return (diff_po_re + diff_ne_re)

def revese_REOs(t2d_REOs, nd_REOs):
    t2d_REOs[t2d_REOs==0] = 100  
    nd_REOs[nd_REOs==0] = 100  
    reverse_REOs = t2d_REOs + nd_REOs
    reverse_REOs_index = np.argwhere(abs(reverse_REOs).values==0)
    reverse_REOs_index = reverse_REOs_index.tolist() 
    uni_reverse_REOs_index = []
    for g in reverse_REOs_index:
        if g[0] < g[1]:   
            uni_reverse_REOs_index.append(g)
    
    symbol = reverse_REOs.index.values.tolist()  
    g1 = []
    g2 = []
    for i in uni_reverse_REOs_index:
        g1.append(symbol[i[0]])
        g2.append(symbol[i[1]])
        reverse_REOs_list = pd.DataFrame({"g1":g1, "g2":g2})
    return reverse_REOs_list

In [38]:
def encoding_matrix(exp,gene_pair):
    sample_num = exp.shape[1]
    gene_pair_num = gene_pair.shape[0] 
    gene_index = gene_pair.apply(lambda x: x[0]+"|"+x[1], axis=1)
    encoding_data = pd.DataFrame(index=exp.columns, columns=gene_index)
    for sample_name in exp.columns:
        for gene_name in gene_index:
            gene_name1, gene_name2 = gene_name.split("|")
            if exp.loc[gene_name1, sample_name] > exp.loc[gene_name2, sample_name]:
                encoding_data.loc[sample_name, gene_name] = 1
            else:
                encoding_data.loc[sample_name, gene_name] = -1
    return encoding_data

In [39]:
GSE76894_t2d_exp = GSE76894_train.iloc[:,0:13]
GSE76894_nd_exp = GSE76894_train.iloc[:,13:]
GSE76894_t2d_REOs = GSE76894_t2d_exp.apply(t2d_matrix, axis=1, t2d_exp=GSE76894_t2d_exp,threshold=0.65)
GSE76894_nd_REOs =  GSE76894_nd_exp.apply(nd_matrix, axis=1, nd_exp=GSE76894_nd_exp,threshold=0.65)

In [44]:
GSE76894_t2d_REOs[GSE76894_t2d_REOs == 0] = 100  
GSE76894_nd_REOs [GSE76894_nd_REOs == 0] = 100  
reverse_stable_gene = GSE76894_t2d_REOs + GSE76894_nd_REOs 

reverse_stable_gene_index = np.argwhere(abs(reverse_stable_gene).values == 0)
reverse_stable_gene_index = reverse_stable_gene_index.tolist() 
uni_reverse_stable_gene_index = []
for g in reverse_stable_gene_index:
    if g[0] < g[1]:   
        uni_reverse_stable_gene_index.append(g)
symbol = reverse_stable_gene.index.values.tolist()  
g1 = []
g2 = []
for i in uni_reverse_stable_gene_index:
    g1.append(symbol[i[0]])
    g2.append(symbol[i[1]])
reverse_gene_pair_dic = {"g1":g1,"g2":g2}
GSE76894_revese_REOs = pd.DataFrame(reverse_gene_pair_dic)
GSE76894_revese_REOs

,g1,g2
0,C18orf32,TMSB10
1,EEF1A1,TMSB10
2,FTL,RPS15A
3,FTL,RPL37
4,FTL,RPL23A
...,...,...
238090,FAM154B,NDP
238091,CIITA,DCTN1-AS1
238092,DCTN1-AS1,NRK
238093,DNAI1,LINC00520


In [45]:
GSE164416_t2d_exp = GSE164416_train.iloc[:,0:27]
GSE164416_nd_exp =  GSE164416_train.iloc[:,27:]
GSE164416_t2d_REOs = GSE164416_t2d_exp.apply(t2d_matrix, axis=1, t2d_exp=GSE164416_t2d_exp,threshold=0.65)
GSE164416_nd_REOs =  GSE164416_nd_exp.apply(nd_matrix, axis=1,   nd_exp=GSE164416_nd_exp,threshold=0.65)

In [ ]:
GSE164416_t2d_REOs[GSE164416_t2d_REOs == 0] = 100  
GSE164416_nd_REOs [GSE164416_nd_REOs == 0] = 100  
reverse_stable_gene = GSE164416_t2d_REOs + GSE164416_nd_REOs 

reverse_stable_gene_index = np.argwhere(abs(reverse_stable_gene).values == 0)
reverse_stable_gene_index = reverse_stable_gene_index.tolist() 
uni_reverse_stable_gene_index = []
for g in reverse_stable_gene_index:
    if g[0] < g[1]:   
        uni_reverse_stable_gene_index.append(g)
symbol = reverse_stable_gene.index.values.tolist()  
g1 = []
g2 = []
for i in uni_reverse_stable_gene_index:
    g1.append(symbol[i[0]])
    g2.append(symbol[i[1]])
reverse_gene_pair_dic = {"g1":g1,"g2":g2}
GSE164416_revese_REOs = pd.DataFrame(reverse_gene_pair_dic)
GSE164416_revese_REOs

In [ ]:
GSE54279_t2d_exp = GSE54279_train.iloc[:,0:11]
GSE54279_nd_exp =  GSE54279_train.iloc[:,11:]
GSE54279_t2d_REOs = GSE54279_t2d_exp.apply(t2d_matrix, axis=1, t2d_exp=GSE54279_t2d_exp,threshold=0.65)
GSE54279_nd_REOs =  GSE54279_nd_exp.apply(nd_matrix, axis=1,   nd_exp=GSE54279_nd_exp,threshold=0.65)

In [ ]:
GSE54279_t2d_REOs[GSE54279_t2d_REOs == 0] = 100  
GSE54279_nd_REOs [GSE54279_nd_REOs == 0] = 100  
reverse_stable_gene = GSE54279_t2d_REOs + GSE54279_nd_REOs 

reverse_stable_gene_index = np.argwhere(abs(reverse_stable_gene).values == 0)
reverse_stable_gene_index = reverse_stable_gene_index.tolist() 
uni_reverse_stable_gene_index = []
for g in reverse_stable_gene_index:
    if g[0] < g[1]:   
        uni_reverse_stable_gene_index.append(g)
symbol = reverse_stable_gene.index.values.tolist()  
g1 = []
g2 = []
for i in uni_reverse_stable_gene_index:
    g1.append(symbol[i[0]])
    g2.append(symbol[i[1]])
reverse_gene_pair_dic = {"g1":g1,"g2":g2}
GSE54279_revese_REOs = pd.DataFrame(reverse_gene_pair_dic)
GSE54279_revese_REOs

In [ ]:
GSE76894_gene_pair = np.array(GSE76894_revese_REOs.apply(lambda x: x[0]+"|"+x[1], axis=1))  
GSE164416_gene_pair = np.array(GSE164416_revese_REOs.apply(lambda x: x[0]+"|"+x[1], axis=1))  
GSE54279_gene_pair = np.array(GSE54279_revese_REOs.apply(lambda x: x[0]+"|"+x[1], axis=1))
com_gene_pair = reduce(np.intersect1d, [GSE76894_gene_pair,GSE164416_gene_pair,GSE54279_gene_pair])
# reverse_gene_pair_symbol.to_csv("/home/xqxie/Task/02_stable_reverse_gene_pair/6_ESWA_review/com_65%_gene_pair.csv",index = False)
com_gene_pair

In [ ]:
GSE76894_encoding = encoding_matrix(GSE76894_train,com_gene_pair)
t2d_num = [1]*13
nd_num = [0]*59
label = t2d_num + nd_num
GSE76894_encoding.insert(0,'label',label)


GSE164416_encoding = encoding_matrix(GSE164416_train,com_gene_pair)
t2d_num = [1]*27
nd_num = [0]*12
label = t2d_num + nd_num
GSE164416_encoding.insert(0,'label',label)
GSE164416_encoding

GSE54279_encoding = encoding_matrix(GSE54279_train,com_gene_pair)
t2d_num = [1]*11
nd_num = [0]*31
label = t2d_num + nd_num
GSE54279_encoding.insert(0,'label',label)
GSE54279_encoding

merge_train_encoding = pd.concat([GSE76894_encoding, GSE164416_encoding, GSE54279_encoding], axis=0)

x_train = train_data.iloc[:,:6] 
y_train = train_data.iloc[:,6] 
x_train

x_np = np.array(x_train)
y_np = np.array(y_train)